In [1]:
import pymongo
from pymongo import MongoClient, GEOSPHERE
import shapely.geometry
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
import os
import requests
import json
from functools import reduce
import operator
import geopandas as gpd
import cartoframes
from cartoframes.viz import Map, Layer, popup_element

### 1. Connecting with Foursquare Api.

In [2]:
load_dotenv()
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [3]:
office_location = {'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}
print (office_location)

{'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}


### 2. Defining vegan restaurants' query

In [4]:
url_query = 'https://api.foursquare.com/v2/venues/explore'
vegan = "4bf58dd8d48988d1d3941735"

In [5]:
# By taking information from Foursquare's API, we are looking for vegan restaurants around our office in a radius of 500m.
parameters = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "query":f"vegans",
                "radius":500}

In [6]:
resp = requests.get(url= url_query, params = parameters)
data = json.loads(resp.text)

In [7]:
decoding_data = data.get("response")
decoded = decoding_data.get("groups")[0]
vegan = decoded.get("items")

In [8]:
map_ = ["venue","name"]
m_latitude = ["venue","location","lat"]
m_longitude = ["venue","location","lng"]

In [9]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [10]:
print(getFromDict(vegan[0],map_))

Beyond Sushi


In [11]:
vegan_nyc = []
for dic in vegan:
    list1 = {}
    list1["name"] = getFromDict(dic,map_)
    list1["latitud"] = getFromDict(dic,m_latitude)
    list1["longitud"] = getFromDict(dic,m_longitude)
    vegan_nyc.append(list1)

In [12]:
# Conversion on dataframe that are used on maps construction.
df_vegans = pd.DataFrame(vegan_nyc)
df_vegans.head()

,name,latitud,longitud
0,Beyond Sushi,40.763454,-73.977163
1,by CHLOE.,40.758063,-73.978854
2,Van Leeuwen Ice Cream,40.759578,-73.980338
3,Taco Dumbo,40.760295,-73.977430
4,L’Avenue,40.758756,-73.977673


**30 Vegan restaurants** are located in a radius of 500m from the office.

In [13]:
#### Finally, we convert our data frame into a geodataframe.

gdf_vegan = gpd.GeoDataFrame(df_vegans, geometry = gpd.points_from_xy(df_vegans.longitud,df_vegans.latitud))
gdf_vegan.head()

,name,latitud,longitud,geometry
0,Beyond Sushi,40.763454,-73.977163,POINT (-73.97716 40.76345)
1,by CHLOE.,40.758063,-73.978854,POINT (-73.97885 40.75806)
2,Van Leeuwen Ice Cream,40.759578,-73.980338,POINT (-73.98034 40.75958)
3,Taco Dumbo,40.760295,-73.977430,POINT (-73.97743 40.76029)
4,L’Avenue,40.758756,-73.977673,POINT (-73.97767 40.75876)


In [14]:
### Creating the column criteria
# Adding a criteria column
gdf_vegan = gdf_vegan.assign(name = ["Beyond Sushi",
                                        "by CHLOE.",
                                        "Van Leeuwen Ice Cream",
                                        "Taco Dumbo",
                                        "L’Avenue",
                                        "Taco Dumbo",
                                        "Urbanspace W52",
                                        "The Halal Guys",
                                        "The Little Beet",
                                        "Lenwich by Lenny's",
                                        "Le Pain Quotidien",
                                        "Mysttik Masala",
                                        "Museum of Modern Art (MoMA)",
                                        "Fogo de Chão",
                                        "Aldo Sohm Wine Bar",
                                        "Gregorys Coffee",
                                        "Pret A Manger",
                                        "Devon & Blakely",
                                        "Fig & Olive",
                                        "The Modern",
                                        "Black Tap",
                                        "Simon Sips",
                                        "Barilla Restaurants",
                                        "Estiatorio Milos",
                                        "Le Pain Quotidien",
                                        "Cock & Bull British Pub and Eatery",
                                        "Europa Cafe",
                                        "Forty2West",
                                        "Butter Midtown",
                                        "Natureworks"],
                                criteria = ["vegan"]*30)
gdf_vegan.head()

,name,latitud,longitud,geometry,criteria
0,Beyond Sushi,40.763454,-73.977163,POINT (-73.97716 40.76345),vegan
1,by CHLOE.,40.758063,-73.978854,POINT (-73.97885 40.75806),vegan
2,Van Leeuwen Ice Cream,40.759578,-73.980338,POINT (-73.98034 40.75958),vegan
3,Taco Dumbo,40.760295,-73.977430,POINT (-73.97743 40.76029),vegan
4,L’Avenue,40.758756,-73.977673,POINT (-73.97767 40.75876),vegan


In [15]:
# Convert our dataframe on a dict to be able to use it in MongoDB.

In [16]:
gdf_vegan["geometry"] = gdf_vegan["geometry"].apply(lambda x: shapely.geometry.mapping(x))
gdf_vegan[:5]

/usr/local/lib/python3.9/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,geometry,criteria
0,Beyond Sushi,40.763454,-73.977163,"{'type': 'Point', 'coordinates': (-73.97716274...",vegan
1,by CHLOE.,40.758063,-73.978854,"{'type': 'Point', 'coordinates': (-73.9788542,...",vegan
2,Van Leeuwen Ice Cream,40.759578,-73.980338,"{'type': 'Point', 'coordinates': (-73.980338, ...",vegan
3,Taco Dumbo,40.760295,-73.977430,"{'type': 'Point', 'coordinates': (-73.97743043...",vegan
4,L’Avenue,40.758756,-73.977673,"{'type': 'Point', 'coordinates': (-73.97767334...",vegan


In [17]:
type(gdf_vegan.loc[0,"geometry"])

dict

### 3. Export the query to Mongo

In [18]:
client = MongoClient() 
db = client.companies
collection = db.vegan 
collection.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [19]:
data = gdf_vegan.to_dict(orient='records')
data[0]

{'name': 'Beyond Sushi',
 'latitud': 40.76345369164551,
 'longitud': -73.9771627466244,
 'geometry': {'type': 'Point',
  'coordinates': (-73.9771627466244, 40.76345369164551)},
 'criteria': 'vegan'}

In [20]:
collection.insert_many(data)

### 4. Determine the nearest vegan restaurant

In [21]:
db.vegan

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'vegan')

In [22]:
filt = {"geometry":{"$near":{"type":"Point","coordinates":[40.7592189, -73.9783534]}}}
proyect = {"_id":0,"name":1,"latitud":1,"longitud":1}

result = db.vegan.find(filt,proyect).limit(1)
vegan = list(result)

In [23]:
vegan

[{'name': 'Devon & Blakely',
  'latitud': 40.75560132228682,
  'longitud': -73.97619031651554}]